# 02 — Linear Regression (Closed Form, OLS) — From Scratch

**Goal:** Implement OLS with intercept, compare to scikit-learn.


In [ ]:
import warnings
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing, make_regression
from sklearn.model_selection import train_test_split

def load_regression_data(random_state=42):
    """Return (X, y, feature_names) as numpy arrays.
    Try California Housing; fallback to synthetic if unavailable (e.g., offline).
    """
    try:
        cali = fetch_california_housing(as_frame=True)
        df = cali.frame.copy()
        X = df.drop(columns=["MedHouseVal"]).values
        y = df["MedHouseVal"].values
        feature_names = list(df.drop(columns=["MedHouseVal"]).columns)
    except Exception as e:
        warnings.warn(f"California Housing fetch failed: {e}. Falling back to synthetic make_regression.")
        X, y = make_regression(n_samples=5000, n_features=8, n_informative=6, noise=8.5, random_state=random_state)
        feature_names = [f"x{i}" for i in range(X.shape[1])]
    return X, y, feature_names

def train_val_test_split(X, y, random_state=42):
    # 60/20/20 split: train/val/test
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=random_state)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=random_state)
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

def rmse(y_true, y_pred):
    return float(np.sqrt(np.mean((y_true - y_pred)**2)))

def mae(y_true, y_pred):
    return float(np.mean(np.abs(y_true - y_pred)))

def r2(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return float(1 - ss_res/ss_tot)


In [ ]:
import numpy as np
from numpy.linalg import inv, pinv
from sklearn.linear_model import LinearRegression

# Prepare data with intercept term
X, y, feature_names = load_regression_data()
(X_train, y_train), (X_val, y_val), (X_test, y_test) = train_val_test_split(X, y)

def add_intercept(X):
    return np.c_[np.ones((X.shape[0], 1)), X]

Xtr_i = add_intercept(X_train)
Xval_i = add_intercept(X_val)
Xte_i = add_intercept(X_test)

# TODO: Implement OLS using normal equation with pseudo-inverse for stability.
def ols_fit(Xi, y):
    beta = pinv(Xi) @ y
    return beta

def ols_predict(Xi, beta):
    return Xi @ beta

beta_hat = ols_fit(Xtr_i, y_train)
y_pred_val = ols_predict(Xval_i, beta_hat)

def rmse(y_true, y_pred):
    return float(np.sqrt(np.mean((y_true - y_pred)**2)))

print("From-scratch OLS RMSE (val):", rmse(y_val, y_pred_val))

# Compare to scikit-learn
lr = LinearRegression(fit_intercept=True)
lr.fit(X_train, y_train)
y_pred_val_skl = lr.predict(X_val)
print("Sklearn LinearRegression RMSE (val):", rmse(y_val, y_pred_val_skl))

# TODO: Assert they're close (tolerance ~1e-6 to 1e-4 depending on data scaling).


In [ ]:
# TODO: Plot residuals (y_true - y_pred) vs predicted for both implementations. One plot per cell.
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(y_pred_val, y_val - y_pred_val, s=6)
plt.axhline(0)
plt.title("Residuals vs Pred (From-Scratch OLS)")

plt.figure()
plt.scatter(y_pred_val_skl, y_val - y_pred_val_skl, s=6)
plt.axhline(0)
plt.title("Residuals vs Pred (Sklearn OLS)")
